# Sources

After way too much struggle and googeling, I finally found this:

https://colab.research.google.com/github/luxonis/depthai-ml-training/blob/master/colab-notebooks/Easy_Object_Detection_With_Custom_Data_Demo_Training.ipynb

# While Hack

In [ ]:
while True:
    pass

# Install Libraries

In [ ]:
# %%capture

#After this cell executes runtime will restart, ignore the crash message, continue running cells starting with the one below

# a specific numpy version is installed
!pip install numpy==1.17.5;

# the runtime has to be crased (to be restarted to laod the new version)

# !pip install tensorflow-gpu==1.15.0;
import os
os.kill(os.getpid(), 9)


In [ ]:
# use the tensorflow version 1 (instead of 2 default)
%tensorflow_version 1.x

!pip install tf_slim

# Select a base model

In [ ]:
# For faster training time, images should be resized to 300x300 and then annotated
# Images should contain the objects of interest at various scales, angles, lighting conditions, locations

# a step means using a single batch of data
# larger batch, less steps required
num_steps = 1000

# number of evaluation steps
num_eval_steps = 50

# batch size 24 is a setting that generally works well
# can be changed higher or lower, depending on model size and available VRAM 
# here some general information is stored in a dictionary
# one of the models can then be selected
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 32 #24
    },
        'ssd_resnet_50': {
        'model_name': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03',
        'pipeline_file': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config',
        'batch_size': 8
    },
        'ssd_inception_v2': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'pipeline_file': 'ssd_inception_v2_coco.config',
        'batch_size': 32
    }
}

# select one of the models to be used as the base model
# using transfer learning, the model will be retrained to suit your specific needs
selected_model = 'ssd_mobilenet_v2'
#selected_model = 'ssd_resnet_50'
#selected_model = 'ssd_inception_v2'

# name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `object_detection_demo_flow` repository

In [ ]:
repo_url = 'https://github.com/GotG/object_detection_demo_flow'
import os

# escape the notebook with %
%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

# escape the notebook with !
# clone the repository
!git clone {repo_url}
%cd {repo_dir_path}

# get new changes
!git pull

## Upload your files

### Old option: Upload the files every time

In [ ]:
from google.colab import files

%cd /content
print("Upload the allimages.zip file")
files.upload()

!unzip allimages.zip -d ./allimages

In [ ]:
%cd /content

# clear original images
!rm object_detection_demo_flow/test/*
!rm object_detection_demo_flow/data/images/train/*
!rm object_detection_demo_flow/data/images/test/*
!rm object_detection_demo_flow/data/images/final_test/*

# instead, copy my images :D
!cp allimages/* object_detection_demo_flow/data/images/train/
!cp allimages/* object_detection_demo_flow/data/images/test/
!cp allimages/* object_detection_demo_flow/data/images/final_test/

### New option: have dataset on Google Drive

#### Prepare dataset

One time setup.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
"""
%cd /content

# clear original images
!rm object_detection_demo_flow/test/*
!rm object_detection_demo_flow/data/images/train/*
!rm object_detection_demo_flow/data/images/test/*
!rm object_detection_demo_flow/data/images/final_test/*

!mkdir allimages

import glob, os
from shutil import copyfile
os.chdir("/content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/labels")
for file in glob.glob("*.xml"):
    #print(file)
    copyfile(f"/content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/allimages/{file[:-4]}.jpg", f"/content/allimages/{file[:-4]}.jpg")
!cp /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/labels/* /content/allimages/
"""

In [ ]:
"""
%cd /content
!mkdir /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/1000images

# instead, copy my images :D
!cp allimages/* object_detection_demo_flow/data/images/train/
!cp allimages/* object_detection_demo_flow/data/images/test/
!cp allimages/* object_detection_demo_flow/data/images/final_test/
!cp allimages/* /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/1000images/
"""

### Load the dataset

First mount Google Drive, where the dataset lives.

The connection to Google Drive is slower than to the runtime itself.
That is why the images are copied over initially once.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

%cd /content
!mkdir allimages
!cp /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/1000images/* /content/allimages/

# clear original images
!rm object_detection_demo_flow/test/*
!rm object_detection_demo_flow/data/images/train/*
!rm object_detection_demo_flow/data/images/test/*
!rm object_detection_demo_flow/data/images/final_test/*

# instead, copy my images :D
!cp allimages/* object_detection_demo_flow/data/images/train/

# note that it is really bad practice to have the same images for training and testing
# but we want a working result as fast as possible
# the accuracy does not matter at first
!cp allimages/* object_detection_demo_flow/data/images/test/
!cp allimages/* object_detection_demo_flow/data/images/final_test/

## Clone TF models which contains the Object Detection API; also install the required dependencies


In [ ]:
%%capture
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

## Prepare `tfrecord` files



In [ ]:
%%capture
%cd {repo_dir_path}

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

In [ ]:
test_record_fname = '/content/object_detection_demo_flow/data/annotations/test.record'
train_record_fname = '/content/object_detection_demo_flow/data/annotations/train.record'
label_map_pbtxt_fname = '/content/object_detection_demo_flow/data/annotations/label_map.pbtxt'

## Download the Model, depending on selection

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# untar the downloaded file
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# remove the tar
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)

os.rename(MODEL, DEST_DIR)
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

In [ ]:
#TF pretrained model checkpoint
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

# check if the configuration file exists
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

# function to get the number of classes
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    # load the label map
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    # get the categories ffrom the map
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    # index the categories
    category_index = label_map_util.create_category_index(categories)
    # return the length (number of unique ids)
    return len(category_index.keys())

### ResNet 50 handling

In case the ResNet 50 was selected as the base model, an updated configuration file has to be used.

Read the full documentation on [GitHub](https://github.com/MxFxM/TGMB) to find out what to change.

In [ ]:
if selected_model == 'ssd_resnet_50':
    print("Upload ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config now")
    from google.colab import files
    files.upload()
    !cp /content/models/research/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config /content/models/research/object_detection/samples/configs/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config


## Overwrite default pipeline values

In [ ]:
import re
iou_threshold = 0.50
num_classes = get_num_classes(label_map_pbtxt_fname)

# reat the old file
with open(pipeline_fname) as f:
    s = f.read()

# by substitutions, enter the new values
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    # Set number of classes num_classes.
    s = re.sub('iou_threshold: [0-9].[0-9]+',
               'iou_threshold: {}'.format(iou_threshold), s)
    
    f.write(s)

# Train the model

## Start the training

In [ ]:
# note that training is stored in /content/models/research/training/
model_dir = 'training/'

# os.makedirs(model_dir, exist_ok=True)

# this is all, the rest is handled by the API
!python -W ignore /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

## Check the output in the model directory

In [ ]:
# model dir check for the trained model
!ls {model_dir}

## Export a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
#clean output_directory if necessary to start fresh:

# !rm -rf /content/object_detection_demo/fine_tuned_model/ 
# os.makedirs('/content/object_detection_demo_flow/fine_tuned_model/', exist_ok=True)

The frozen inference graph "freezes" the model at a certain state.
This state can be used to compile the blob file with OpenVINO, or as a general method of storing the model.

In [ ]:
%%capture
import re
import numpy as np

output_directory = './fine_tuned_model'

# list all files in the checkpoint directory
lst = os.listdir(model_dir)

# keep only the ones that end on .meta
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]

# find the number of steps (it is in the filename of the checkpoint)
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])

# get the chekcopoint with the largest number of steps
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

# export that checkpoint as the frozen inference graph
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
import os
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")

# check for the file
# if it does not exist, the export has failed
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

# !ls -alh {pb_fname}

# Intel OpenVINO toolkit Installation and Validations 2021

## Installation

No questions asked for the OpenVINO installation. It works, that is all that is important.

In [ ]:
!cd /content

In [ ]:
print("This will take a while...")
!wget -O openvino_key https://apt.repos.intel.com/openvino/2021/GPG-PUB-KEY-INTEL-OPENVINO-2021?elq_cid=6770273_ts1607381885691&erpm_id=9830841_ts1607381885691&elq_cid=6770273_ts1607381960247&erpm_id=9830841_ts1607381960247 
!apt-key add openvino_key
#!apt-key -q list
!echo "deb https://apt.repos.intel.com/openvino/2021 all main" | tee /etc/apt/sources.list.d/intel-openvino-2021.list
!apt update -q
!apt-cache search intel-openvino-dev-ubuntu18
!apt-get install intel-openvino-dev-ubuntu18-2021.2.200 -y -q
!pip install -U --no-deps --quiet openvino

#unknown bug - this is just a patch
!cp /opt/intel/openvino_2021/deployment_tools/inference_engine/external/tbb/lib/libtbb.so /usr/lib/x86_64-linux-gnu/libtbb.so
!cp /opt/intel/openvino_2021/deployment_tools/inference_engine/external/tbb/lib/libtbb.so.2 /usr/lib/x86_64-linux-gnu/libtbb.so.2
!ldconfig
print("Installation Completed...")
#%env we can set environmental variable with this

#Run the validation
!ls /opt/intel/
#Run the Validation Demo code.
demo_cmd = "/opt/intel/openvino_2021/deployment_tools/demo/demo_squeezenet_download_convert_run.sh"
import os
import subprocess
import shutil
output = subprocess.check_output(demo_cmd, shell=True)
print (output.decode('utf-8'))     

## Convert to OpenVINO IR format

There is a small manual change in the support file, necessary when compiling some of the models.

In [ ]:
%cd /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/
#openvino fixes: edit 
# Read in the file, make sure the .json corresponds to the model!!!
with open('ssd_v2_support.json', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"Postprocessor/ToFloat"', '"Postprocessor/Cast_1"')

# Write the file out again
with open('ssd_v2_support.json', 'w') as file:
  file.write(filedata)

Each model has its own configuration used for the model optimizer.

To find your options (arguments), head over to the [OpenVINO Model Zoo](https://github.com/openvinotoolkit/open_model_zoo) and check the model.yml file for your specific base model.

In [ ]:
source_cmd= "/opt/intel/openvino_2021/bin/setupvars.sh"

%cd "/content/models/research/fine_tuned_model/"

# Setup model optimizer command ...
ir_name = "bee_detection"

# data type
ir_data_type = "FP16"

# output directory
ir_out_dir = f"/content/IR_models/{ir_data_type}"

# input shape (might be overwritten depending on the base model)
ir_input_shape = "[1,300,300,3]"

model_fname = "/content/models/research/fine_tuned_model/"

if selected_model == 'ssd_mobilenet_v2':
    !source /opt/intel/openvino_2021/bin/setupvars.sh && \
        python /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py \
        --input_model frozen_inference_graph.pb \
        --input_shape [1,300,300,3] \
        --tensorflow_object_detection_api_pipeline_config pipeline.config \
        --reverse_input_channels \
        --output_dir {ir_out_dir} \
        --data_type FP16 \
        --input image_tensor \
        --output detection_classes,detection_scores,detection_boxes,num_detections \
        --tensorflow_use_custom_operations_config /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json

if selected_model == 'ssd_resnet_50':
    !source /opt/intel/openvino_2021/bin/setupvars.sh && \
        python /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py \
        --input_model frozen_inference_graph.pb \
        --input_shape [1,640,640,3] \
        --tensorflow_object_detection_api_pipeline_config pipeline.config \
        --reverse_input_channels \
        --output_dir {ir_out_dir} \
        --data_type FP16 \
        --input image_tensor \
        --output detection_classes,detection_scores,detection_boxes,num_detections \
        --tensorflow_use_custom_operations_config /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json  

## Convert OpenVINO IR to Blob format 

Then the IR file can be converted to the blob file.

The number of shaves can be adjusted.
The OAK-D has 16 shaves in total, 13 can be used when the main camera is in 1080P mode and 10 when the camera runs in 4K mode.
The remaining shaves can be split up into the models that have to run simultaneously on the same pipeline.

In [ ]:
!cp /opt/intel/openvino_2021.2.200/deployment_tools/inference_engine/lib/intel64/libmyriadPlugin.so /usr/lib/x86_64-linux-gnu/libmyriadPlugin.so
#!source /opt/intel/openvino_2021/bin/setupvars.sh && /opt/intel/openvino_2021/deployment_tools/inference_engine/lib/intel64/myriad_compile -m {ir_out_dir}/frozen_inference_graph.xml -ip U8 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4
!source /opt/intel/openvino_2021/bin/setupvars.sh && /opt/intel/openvino_2021/deployment_tools/inference_engine/lib/intel64/myriad_compile -m {ir_out_dir}/frozen_inference_graph.xml -ip U8 -VPU_NUMBER_OF_SHAVES 10 -VPU_NUMBER_OF_CMX_SLICES 10

## Download blob file

In [ ]:
from google.colab import files

#download the blob file to local directory
files.download(f"{ir_out_dir}/frozen_inference_graph.blob")